In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

In [ ]:
#single generation

from transformers import AutoTokenizer, AutoModelForCausalLM

# Model ve tokenizer yollarını belirtin
model_path = "/content/sample_data/models/"
tokenizer_path = "/content/sample_data/models/"

# Tokenizer ve modeli yerel yoldan yükleme
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Modeli direkt olarak kullanın (herhangi bir .to veya .half çağrısı yapmadan)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Verilen cümlenin orijinal anlamı koruyup, farklı kelimeler kullanarak yeniden ifade et.", # instruction
            "bir sosyal hizmet uzmanı eski bir müşteriyle buluşabilir mi?", # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors="pt").to('cuda')

# Modeli kullanarak token üretimi
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Çıktıları decode et
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result)


In [ ]:
#excel test generation

from transformers import TextStreamer, AutoTokenizer, AutoModelForCausalLM
import torch
import openpyxl

# TextStreamer nesnesini oluşturun

# Model ve tokenizer yollarını belirtin
model_path = "/content/sample_data/models/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
text_streamer = TextStreamer(tokenizer)

# Excel dosyasını yükleyin
wb = openpyxl.load_workbook('exceltest600.xlsx')
sheet = wb.active

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


# Excel dosyasındaki verileri kullanarak modeli çalıştırın
for row in range(2, sheet.max_row + 1):
    instruction = "Verilen cümlenin orijinal anlamı koruyup, farklı kelimeler kullanarak yeniden ifade et."
    input_text = sheet[f'A{row}'].value

    # Model için girdileri hazırlayın
    inputs = tokenizer(
        [
            alpaca_prompt.format(instruction, input_text, "")
        ],
        return_tensors="pt"
    ).to("cuda")

# Modeli çalıştırın ve sonuçları alın
    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

    # Sonuçları decode edin ve Excel dosyasına yazın
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # İlk olarak, sadece modelin ürettiği çıktıyı (output) ayıklayın
    output_only = result.split(input_text)[-1].strip()
    final_output = output_only.replace('Response:', '').strip()

    # Son olarak, temizlenmiş çıktıyı Excel dosyasının C sütununa yazdırın
    sheet[f'C{row}'].value = final_output



# Excel dosyasını kaydedin
wb.save('testresult.xlsx')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Verilen cümlenin orijinal anlamı koruyup, farklı kelimeler kullanarak yeniden ifade et.

### Input:
Hey, benim için de zevkli değil.

### Response:
Hey, benim için de zevkli değildi.<eos>
<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Verilen cümlenin orijinal anlamı koruyup, farklı kelimeler kullanarak yeniden ifade et.

### Input:
Belki Krill o kadar da kötü biri değildir.

### Response:
Belki Krill o kadar da kötü değildir.<eos>
<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Verilen cümlenin orijinal anlamı koruyup, farklı kelimeler kullanara